In [75]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder




train = pd.read_csv('../input/titanic/train.csv')
test = pd.read_csv('../input/titanic/test.csv')

train_new = train.copy()
test_new = test.copy()

**9.1 - Feature Selection**

In [76]:
def Mi_score(features, target):
    MI_sc = mutual_info_classif(features,target,random_state = 0)
    MI_sc = pd.Series(MI_sc,index = features.columns)
    MI_sc = MI_sc.sort_values(ascending = True)
    
    plt.bar(np.arange(len(MI_sc)), MI_sc)
    
    return MI_sc

Função minha do EX 6.

In [77]:
train_new.info()

In [78]:
train_new['Age'].fillna(method='ffill', inplace=True)
#test_new['Age'].fillna(method='ffill', inplace=True)

train_new['Embarked'].fillna(method='ffill', inplace=True)

#test_new['Age_Intervals'] = pd.cut(test_new['Age'], bins=5)
#train_new['Age_Intervals'] = pd.cut(train_new['Age'], bins=5)

train_new_Y = train_new['Survived']
train_new_X = train_new.drop(columns=['Survived', 'Name', 'PassengerId', 'Ticket', 'Cabin'])


def indent(db, typ):
    p = (db.dtypes == typ)
    return list(p[p].index)

s = indent(train_new_X, 'object')

def ONEHOT(db, obj):
    for x in obj:
        if db[x].dtypes == 'object':
            label_encod = LabelEncoder()
            label_bd = label_encod.fit_transform(db[x])
            
            Onehotencod = OneHotEncoder(sparse=False)
            db[x] = Onehotencod.fit_transform(label_bd.reshape(len(label_bd),1))
            
        elif (db[x].dtypes != 'object') and (db[x].dtypes != 'float64') and (db[x].dtypes != 'int64'):
            label_encod = LabelEncoder()
            db[x] = label_encod.fit_transform(db[x])
            
ONEHOT(train_new_X, s)

mi_score_result = Mi_score(train_new_X, train_new_Y)

**Criando os gráficos**

In [79]:
fig, pl = plt.subplots(1,2, figsize=(10,5))
pl[0].set_title('Fare X Survived')
sns.histplot(data= train_new, x = 'Fare', kde=True, hue='Survived', ax = pl[0])

pl[1].set_title('Fare')
sns.histplot(data=train_new, x = 'Fare', kde= True, ax = pl[1])

**Segunda parte dos Gráficos**

In [80]:
fig, pl = plt.subplots(2,2, figsize=(15,10))

pl[0,0].set_title('Relação de Idade e Gênero')
sns.histplot(data = train_new, x= 'Age', hue= 'Sex', ax = pl[0,0])

pl[0,1].set_title('Relação de Idade/Sobrevivente e Gênero')
sns.histplot(data = train_new.loc[train_new.Survived == 1], x= 'Age', hue= 'Sex', ax = pl[0,1])

pl[1,0].set_title('Relação de Homens/Sobrevivente e Idade')
sns.histplot(data = train_new.loc[train_new.Sex == 'male'], x= 'Age', hue= 'Survived', ax = pl[1,0])

pl[1,1].set_title('Relação de Mulheres/Sobrevivente e Idade')
sns.histplot(data = train_new.loc[train_new.Sex == 'female'], x= 'Age', hue= 'Survived', ax = pl[1,1])

**9.2 - Feature Construction**

In [81]:
train_new['Title'] = train_new['Name'].str.split(',', expand = True)[1].str.split('.', expand = True)[0]

train_new['Family'] = train_new['SibSp'] + train_new['Parch']

train_new['Alone'] = train_new['Family'].apply(lambda x: True if x ==0 else False)
train_new.drop(columns=['SibSp', 'Parch', 'Cabin'], inplace = True)

train_new.info()

**9.3 - Feature Extraction**

In [82]:
fig,pl = plt.subplots(1,2, figsize=(10,5))
pl[0].set_title('Fare sem limitações')
sns.scatterplot(data = train_new, x = 'Age', y='Fare', ax = pl[0])

#LIMITANDO O FARE
index = train_new.loc[train_new['Fare'] >= 500].index
train_new.drop(index, inplace = True)

pl[1].set_title('Fare com limitações')
sns.scatterplot(data = train_new, x = 'Age', y='Fare', ax = pl[1])


In [83]:
from sklearn.cluster import KMeans

new_train = train_new.loc[:,['Age','Fare','Survived']]
kmeans = KMeans(n_clusters = 6)
new_train['Cluster'] = kmeans.fit_predict(new_train)


fig,pl = plt.subplots(1,2,figsize = (15,10))

sns.scatterplot(data=new_train, x='Fare',y='Age',hue='Cluster', ax = pl[0])

sns.barplot(data=new_train, x= 'Cluster', y='Survived', ax = pl[1])

In [84]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
df = pd.DataFrame()

for col in ['Age', 'Fare']:
    df[[col]] = new_train[[col]]
    df[[col]] = mms.fit_transform(df[[col]])

fig, pl = plt.subplots(1,2,figsize=(10,5))

sns.histplot(data = df, x = 'Age', ax = pl[0])
sns.histplot(data = df, x = 'Fare', ax = pl[1])

**PCA**

In [85]:
from sklearn.decomposition import PCA

pca = PCA()

pca_transformed = pca.fit_transform(df)

names_convert = ['PCA1', 'PCA2']

df_PCA = pd.DataFrame(pca_transformed, columns=names_convert)

**9.4 - Target Encoding**

In [86]:
from category_encoders import MEstimateEncoder

mee = MEstimateEncoder(cols=['Title'], m=5)

df_title_encoder = mee.fit_transform(train_new['Title'], train_new['Survived'])

fig, pl = plt.subplots(figsize=(20,10))

sns.histplot(data=train_new['Survived'], kde=False, stat='density')
sns.kdeplot(data=df_title_enconder['Title'])

pl.set_xlabel('Sobreviventes')
pl.legend(labels=['Título', 'Sobreviventes'])